# Tác vụ 1a: Tạo văn bản

Trong sổ tay này, bạn sẽ tìm hiểu cách sử dụng Mô hình ngôn ngữ lớn (LLM) để tạo email hồi đáp khách hàng đã phản hồi tiêu cực về chất lượng dịch vụ khách hàng của kỹ sư hỗ trợ. Trong sổ tay này, bạn sẽ tạo một email bày tỏ lời cảm ơn dựa trên email trước đó của khách hàng. Bạn sử dụng mô hình Amazon Titan bằng API Amazon Bedrock với ứng dụng Boto3.

Câu lệnh dùng trong tác vụ này được gọi là câu lệnh không cần mẫu (zero-shot). Trong câu lệnh không cần mẫu (zero-shot), bạn dùng ngôn ngữ đơn giản để mô tả tác vụ hoặc đầu ra mong muốn cho mô hình ngôn ngữ. Sau đó, mô hình sẽ sử dụng kiến thức đã được đào tạo trước và các khả năng của mình để tạo phản hồi hoặc hoàn thành tác vụ mà chỉ dựa trên câu lệnh được cung cấp.

#### Kịch bản
Bạn đóng vai Bob, Giám đốc Dịch vụ khách hàng của AnyCompany. Một số khách hàng đang không hài lòng với dịch vụ khách hàng của công ty và phản hồi tiêu cực về dịch vụ của các kỹ sư hỗ trợ khách hàng. Bây giờ, bạn muốn gửi lời xin lỗi về chất lượng dịch vụ kém đến những khách hàng đó và lấy lại lòng tin của họ. Bạn cần sự trợ giúp của LLM để tạo ra một lượng lớn email phù hợp với con người và được cá nhân hóa dựa trên cảm xúc của khách hàng trong các email trao đổi trước đó.

## Tác vụ 1a.1: Thiết lập môi trường

Trong tác vụ này, bạn sẽ thiết lập môi trường của mình.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Tác vụ 1a.2: Tạo văn bản

Trong tác vụ này, bạn sẽ chuẩn bị đầu vào cho dịch vụ Amazon Bedrock để tạo email.

In [ ]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, AnyCompany to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""

In [ ]:
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":8192,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

Tiếp theo, bạn sẽ sử dụng mô hình Amazon Titan.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> ** Lưu ý:** Amazon Titan hỗ trợ một cửa sổ ngữ cảnh gồm khoảng 4 nghìn mã token và chấp nhận những tham số sau:
- `inputText`: Câu lệnh cho LLM
- `textGenerationConfig`: Đây là những tham số mà mô hình sẽ xem xét trong khi tạo đầu ra.

API Amazon Bedrock sẽ cung cấp cho bạn một API `invoke_model` chấp nhận những phần tử sau:
- `modelId`: Đây là ARN của mô hình cho các mô hình nền tảng khác nhau có trong Amazon Bedrock
- `accept`: Loại yêu cầu đầu vào
- `contentType`: Loại nội dung của đầu ra
- `body`: Một chuỗi json bao gồm câu lệnh và các cấu hình

Tham khảo [tài liệu](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) để biết các mã mô hình tạo văn bản có sẵn.

## Tác vụ 1a.3: Gọi mô hình ngôn ngữ lớn Amazon Titan

Trong tác vụ này, bạn sẽ khám phá cách mô hình tạo đầu ra dựa trên câu lệnh đã tạo trước đó.

### Tạo đầu ra hoàn chỉnh

Mô hình Amazon Titan sẽ tạo email này bằng cách tìm hiểu yêu cầu đầu vào và vận dụng hiểu biết vốn có của mô hình về các phương thức khác nhau. Yêu cầu gửi tới API có tính đồng bộ và sẽ chờ mô hình tạo toàn bộ đầu ra.

In [ ]:
#invoke model
modelId = 'amazon.titan-text-express-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [ ]:
# The relevant portion of the response begins after the first newline character
# Below we print the response beginning after the first occurence of '\n'.

email = outputText[outputText.index('\n')+1:]
print(email)


### Tạo đầu ra truyền trực tuyến

Bedrock cũng hỗ trợ truyền trực tuyến đầu ra dưới dạng các đoạn khi mô hình tạo đầu ra. Email này được tạo bằng cách gọi mô hình có tùy chọn truyền trực tuyến. `invoke_model_with_response_stream` sẽ trả về một `ResponseStream` để bạn đọc.

In [ ]:
# invoke model with response stream
output = []
try:
    
    response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

Phương pháp truyền trực tuyến kèm phản hồi giúp bạn nhanh chóng có được đầu ra của mô hình và cho phép dịch vụ hoàn tất đầu ra đó trong khi bạn đọc. Phương pháp này sẽ hữu ích trong những trường hợp sử dụng mà bạn yêu cầu mô hình tạo ra các đoạn văn bản dài hơn. Sau đó, bạn có thể kết hợp tất cả các đoạn được tạo để tạo thành đầu ra hoàn chỉnh và dùng cho trường hợp sử dụng của mình. 

In [ ]:
#combine output chunks
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)


Sau khi thử nghiệm việc sử dụng SDK boto3, bạn đã nắm được những điểm cơ bản về API Amazon Bedrock. Nhờ sử dụng API này, bạn đã thấy được ứng dụng của việc tạo email để trả lời phản hồi tiêu cực của khách hàng.

### Tự thực hiện
- Thay đổi câu lệnh theo trường hợp sử dụng cụ thể của bạn và đánh giá đầu ra của các mô hình khác nhau.
- Thử nghiệm với độ dài mã token để nắm được độ trễ và khả năng phản hồi của dịch vụ.
- Áp dụng các nguyên tắc tạo câu lệnh khác nhau để có được đầu ra tốt hơn.

### Dọn dẹp

Bạn đã hoàn thành sổ tay này. Để chuyển sang phần tiếp theo của phòng thực hành, hãy làm như sau:

- Đóng tệp sổ tay này và tiếp tục với **Task1b.ipynb**.